In [ ]:
# !pip install sqlalchemy
# !pip install psycopg2

### Verificando duplicados

In [ ]:
import pandas as pd
df = pd.read_csv(r"C:\digital college\DA18\PYTHON\Projeto_Unidade_3\_4_Etapa_ETL\merge_csv\merge_colunas_produtos_csv.csv")
duplicados = df.duplicated(subset=["Produto/Unidade"]).sum()
print(duplicados)

### INSERINDO PRODUTOS NO BANCO

In [3]:
import sys
import os
import csv
import psycopg2
# Adiciona o caminho absoluto da pasta _9_funcoes ao Python
caminho_funcoes = r'C:\digital college\DA18\PYTHON\Projeto_Unidade_3\_9_funcoes'
sys.path.append(caminho_funcoes)

# Agora os imports devem funcionar
from conectar_banco import conectar_banco
from atualizar_macrogrupo import atualizar_macrogrupo
from fechar_conexao import fechar_conexao

def importar_produtos():
    conexao, cursor = conectar_banco()
    if not conexao:
        return False

    try:
        with open(r"C:\digital college\DA18\PYTHON\Projeto_Unidade_3\_4_Etapa_ETL\merge_csv\merge_colunas_produtos_csv.csv", encoding="utf-8") as arquivo:
            for linha in csv.reader(arquivo):
                if linha:
                    cursor.execute("INSERT INTO datawarehouse.dim_produtos (prod_und) VALUES (%s)", 
                                 (linha[0].strip(),))
        conexao.commit()
        return True
    except Exception as e:
        print(f"Erro: {e}")
        conexao.rollback()
        return False
    finally:
        fechar_conexao(conexao, cursor)

if __name__ == "__main__":
    if importar_produtos():
        print("Sucesso!")
    else:
        print("Falha na importação")

✓ Conexão estabelecida com sucesso
✓ Conexão encerrada com sucesso
Sucesso!


### INSERINDO NIVEL DE COMERCIALIZAÇÃO

In [ ]:
import sys
import os
import csv
import psycopg2
# Adiciona o caminho absoluto da pasta _9_funcoes ao Python
caminho_funcoes = r'C:\digital college\DA18\PYTHON\Projeto_Unidade_3\_9_funcoes'
sys.path.append(caminho_funcoes)

# Agora os imports devem funcionar
from conectar_banco import conectar_banco
from atualizar_macrogrupo import atualizar_macrogrupo
from fechar_conexao import fechar_conexao

# Abrir o CSV
def importar_nivel_comercializacao():
    conexao, cursor = conectar_banco()
    if not conexao:
        return False
    try:
        with open(r"C:\digital college\DA18\PYTHON\Projeto_Unidade_3\_4_Etapa_ETL\merge_csv\merge_colunas_nivel_comercializacao_csv.csv", encoding="utf-8") as arquivo_csv:
            leitor = csv.reader(arquivo_csv)

            next(leitor)  # Ignora o cabeçalho

            for linha in leitor:
                if linha:  # ignora linhas vazias
                    tipo_de_comercializacao = linha[0].strip()
                    cursor.execute("INSERT INTO datawarehouse.nivel_comercializacao (tipo_de_comercializacao) VALUES (%s)", (tipo_de_comercializacao,))
        conexao.commit()
        return True
    except Exception as e:
        print(f"Erro: {e}")
        conexao.rollback()
        return False
    finally:
        fechar_conexao(conexao, cursor)

if __name__ == "__main__":
    if importar_nivel_comercializacao():
        print("Sucesso!")
    else:
        print("Falha na importação")


### inserir update na tabela produtos os id dos macroprodutos

In [5]:
import sys
import os

# Adiciona o caminho absoluto da pasta _9_funcoes ao Python
caminho_funcoes = r'C:\digital college\DA18\PYTHON\Projeto_Unidade_3\_9_funcoes'
sys.path.append(caminho_funcoes)

# Agora os imports devem funcionar
from conectar_banco import conectar_banco
from atualizar_macrogrupo import atualizar_macrogrupo
from fechar_conexao import fechar_conexao

print("Funções importadas com sucesso!")
def main():
    # Conecta ao banco
    conexao, cursor = conectar_banco()
    if not conexao:
        return

    # Dados manuais (você edita aqui)
    categorias = [(['MURUMURU%','PINHÃO%','GUARANÁ%', 'MACAÚBA%','MAMONA%','CANA%','MACAÚBA','COCO%','PESSEGO%','PÊSSEGO%','LICURI%','MURICI%','ABACATE%','GOIABA%','GRAVIOLA%',
                'ACEROLA%','CAJU%', 'AÇAÍ%','BURITI%', 'ABACAXI%','BANANA%','BARU%','JUÇARA%','LARANJA%','MAÇÃ%','LIMÃO%','JACA%',
                'MAMÃO%', 'MANGA%', 'MARACUJÁ%','PEQUI%','PINHA%','TANGERINA%', 'UMBU%', 'UVA%'], 1),
                (['FAVA%','CANOLA%','CENTEIO%','ARROZ%','AVEIA%','CEVADA%','FEIJÃO%','MILHO%','FLOCOS%','MILHETO%','TRITICALE%','TRIGO%',
                'SOJA%','AVEIA%','CEVADA%','SORGO%'], 2),
                (['BATATA%','MANDIOCA%','CARÁ%','INHAME%','MANDIOCA%','RAIZ%'],3),
                (['BEZERRO%','NOVILHO%','BOI%','CARNE%','FRANGO%','SUÍNO%','BAGRE%','CAÇÃO%','VACA%','CAPRINO%','SARDA%', 'SARDINHA%', 'AMARELA%',
                  'TAMBAQU%','PACU%','JARAQUI%','MATRINX%','PEROÁ%','%AMARELA%','CURIMATÃ%','DOURADO%','PIAU%','CORVINA%','PESCADINHA%','BADEJO%','CAMARÃO%','PIRARUCU%'], 4),
                (['LEITE%','QUEIJO%','OVOS%','IOGURTE%','%LACTEA%'],5),
                (['ÓLEO%','AZEITE%','OLEO%'],6),
                (['AÇÚCAR%', 'AÇUCAR%','MEL%','SAL%'], 7),                # Açúcares
                (['FARINHA%','FUBÁ%', 'FÉCULA%','POLVILHO%','MASSA%'], 8),   # Farinhas
                (['CEBOLA%','PUPUNHA%', 'TOMATE%','ALHO%','PIMENTÃO%','QUIABO%','PEPINO%','ABÓBORA%','ABOBRINHA%','ALFACE%','BERINJELA%','MAXIXE%','CEBOLINHA%',
                  'CHUCHU%','ERVA%','COENTRO%','CHEIRO VERDE%','COUVE%','RÚCULA%','PIMENTA DE CHEIRO%','PIMENTA%'], 9),                 # Hortaliças e Legumes
                (['CACAU%','ALGOD%','CASTANHA%','AMÊNDOA%','AMENDOI%','GIRASSOL%','SEMENTE%', 'CARO%'], 10),                     # Castanhas e Sementes
                (['POLPA%','CAFÉ%', 'EXTRATO%', 'SUCO%','BEBIDA%','CANJICA%'], 11),               # Bebidas e Extratos
                (['LATEX%','LÁTEX%','CASULO%','JUTA%','PÓ%','PIAÇAVA%','SISAL%','UBYFOL%','CAL%','BORRACHA%','CERA%',
                'FIBRA%','FARELO%','PHOSMET%','10-30-15%','ALUGUEL%','DIARISTA%','ENERGIA%','SACARIA%','MACARRÃO%','BOLO%','PÃO%',
                'ADMINISTRADOR%','ANÁLISE%','MOTONIVELADORA%','MARTELETE%', 'SALÁRIO%','TIGER%','TERRA%','SIRIUS%','NOMINEE%','NATIVO%',
                'APROACH%','INCRÍVEL%','IRRIGAÇÃO%','KIFIX%','%M2%',], 12)                #Outros
                ]

    # Processa cada categoria
    for padroes, id_grupo in categorias:
        atualizar_macrogrupo(conexao, cursor, padroes, id_grupo)

    # Fecha conexão
    fechar_conexao(conexao, cursor)

if __name__ == "__main__":
    main()

Funções importadas com sucesso!
✓ Conexão estabelecida com sucesso
✓ Macrogrupo 1: 256 registros atualizados
✓ Macrogrupo 2: 228 registros atualizados
✓ Macrogrupo 3: 70 registros atualizados
✓ Macrogrupo 4: 198 registros atualizados
✓ Macrogrupo 5: 96 registros atualizados
✓ Macrogrupo 6: 30 registros atualizados
✓ Macrogrupo 7: 36 registros atualizados
✓ Macrogrupo 8: 194 registros atualizados
✓ Macrogrupo 9: 128 registros atualizados
✓ Macrogrupo 10: 62 registros atualizados
✓ Macrogrupo 11: 96 registros atualizados
✓ Macrogrupo 12: 180 registros atualizados
✓ Conexão encerrada com sucesso


### Inserir data na dim_tempo

In [22]:
import sys
import os
import csv
import psycopg2
# Adiciona o caminho absoluto da pasta _9_funcoes ao Python
caminho_funcoes = r'C:\digital college\DA18\PYTHON\Projeto_Unidade_3\_9_funcoes'
sys.path.append(caminho_funcoes)

# Agora os imports devem funcionar
from conectar_banco import conectar_banco
from atualizar_macrogrupo import atualizar_macrogrupo
from fechar_conexao import fechar_conexao

# Dados dos meses em português
meses_pt = [
    'Janeiro', 'Fevereiro', 'Março', 'Abril',
    'Maio', 'Junho', 'Julho', 'Agosto',
    'Setembro', 'Outubro', 'Novembro', 'Dezembro'
]

# Abrir o CSV
def importar_nivel_comercializacao():
    conexao, cursor = conectar_banco()
    if not conexao:
        return False
    try:
        # Gerar e inserir dados de 2014 a 2023
        for ano in range(2014, 2024):
            for mes_num in range(1, 13):
                ano_mes = ano * 100 + mes_num
                mes = meses_pt[mes_num - 1]
                
                # Comando SQL para inserção
                insert_query = ("""
                    INSERT INTO datawarehouse.dim_data (ano_mes, mes, ano)
                    VALUES (%s, %s, %s)
                """)
                try:
                    cursor.execute(insert_query, (ano_mes, mes, ano))
                    conexao.commit()
                except psycopg2.IntegrityError:
                    conexao.rollback()  # Caso o registro já exista
                    print(f"Registro {ano_mes} já existe. Pulando...")
                
        return True  # Retorna True apenas depois de processar todos os anos e meses
        
    except Exception as e:
        print(f"Erro: {e}")
        conexao.rollback()
        return False
    finally:
        fechar_conexao(conexao, cursor)

if __name__ == "__main__":
    if importar_nivel_comercializacao():
        print("Sucesso!")
    else:
        print("Falha na importação")

✓ Conexão estabelecida com sucesso
✓ Conexão encerrada com sucesso
Sucesso!
